<h2>Predict The Flight Ticket Price Hackathon</h2>
<br>
Features:
<pre>
Airline: The name of the airline.
Date_of_Journey: The date of the journey
Source: The source from which the service begins.
Destination: The destination where the service ends.
Route: The route taken by the flight to reach the destination.
Dep_Time: The time when the journey starts from the source.
Arrival_Time: Time of arrival at the destination.
Duration: Total duration of the flight.
Total_Stops: Total stops between the source and destination.
Additional_Info: Additional information about the flight
Price: The price of the ticket
</pre>

<h2>Importing Libraries</h2>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train=pd.read_excel("/home/nephalem/Desktop/Data Science/Datasets/Flight_Ticket_Participant_Datasets/Data_Train.xlsx")
test=pd.read_excel("/home/nephalem/Desktop/Data Science/Datasets/Flight_Ticket_Participant_Datasets/Test_set.xlsx")

In [4]:
samplesub=pd.read_excel('/home/nephalem/Desktop/Data Science/Datasets/Flight_Ticket_Participant_Datasets/Sample_submission.xlsx')

In [5]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [6]:
train['Route'].nunique()

128

In [7]:
test['Route'].nunique()

100

In [8]:
set(test['Route']).difference(set(train['Route']))

{'BLR → TRV → DEL',
 'BOM → IXC → DEL → HYD',
 'BOM → VGA → TIR → HYD',
 'CCU → IXR → BBI → BOM → BLR'}

<h4>Checking Null Values</h4>

In [9]:
train.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [10]:
train[train['Route'].isna()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [11]:
#Manually Checked it online
train['Total_Stops'].fillna("1 stop",axis=0,inplace=True)

In [12]:
train.drop(['Route'],axis=1,inplace=True)

In [13]:
train.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

In [14]:
test.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

In [15]:
test.drop(['Route'],axis=1,inplace=True)

In [16]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,16:50,21:35,4h 45m,1 stop,No info,13302


In [17]:
price=train['Price']

In [18]:
train.drop(['Price'],axis=1,inplace=True)

<h4>Combining Dataset</h4>

In [19]:
train.shape

(10683, 9)

In [20]:
test.shape

(2671, 9)

In [21]:
combi = train.append(test, ignore_index=True)

In [22]:
combi.shape

(13354, 9)

<h4>Date_of_Journey</h4>


In [23]:
from datetime import date
import calendar

In [24]:
#Converting to datetime format
combi['Date_of_Journey'] = pd.to_datetime(combi['Date_of_Journey'])

In [25]:
combi.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,IndiGo,2019-03-24,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info
1,Air India,2019-01-05,Kolkata,Banglore,05:50,13:15,7h 25m,2 stops,No info
2,Jet Airways,2019-09-06,Delhi,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info
3,IndiGo,2019-12-05,Kolkata,Banglore,18:05,23:30,5h 25m,1 stop,No info
4,IndiGo,2019-01-03,Banglore,New Delhi,16:50,21:35,4h 45m,1 stop,No info


In [26]:
#Extracting Day from the date
combi['Day'] = combi['Date_of_Journey'].dt.day

In [27]:
#Extracting Month from the date
combi['Month']=combi['Date_of_Journey'].dt.month

In [28]:
#Extracting Weekday from the date
combi['Weekday']=combi['Date_of_Journey'].dt.day_name()

In [29]:
combi.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Day,Month,Weekday
0,IndiGo,2019-03-24,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info,24,3,Sunday
1,Air India,2019-01-05,Kolkata,Banglore,05:50,13:15,7h 25m,2 stops,No info,5,1,Saturday
2,Jet Airways,2019-09-06,Delhi,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info,6,9,Friday
3,IndiGo,2019-12-05,Kolkata,Banglore,18:05,23:30,5h 25m,1 stop,No info,5,12,Thursday
4,IndiGo,2019-01-03,Banglore,New Delhi,16:50,21:35,4h 45m,1 stop,No info,3,1,Thursday


In [30]:
combi['Weekday'].value_counts()

Thursday     2855
Friday       2262
Wednesday    1991
Monday       1858
Tuesday      1536
Saturday     1514
Sunday       1338
Name: Weekday, dtype: int64

In [31]:
#Dropping the Date_of_Journey Column
combi.drop(['Date_of_Journey'],axis=1,inplace=True)

<h4>Dep_Time</h4>

In [32]:
#combi['Departure'] = pd.to_datetime(combi['Dep_Time'])
#combi.drop(['Departure'],axis=1,inplace=True)
#combi=combi.assign(Departure=pd.cut(combi.Dep_Time.dt.hour,[0,6,12,18,24],labels=['Night','Morning','Afternoon','Evening']))

In [33]:
combi['Hour'] = combi["Dep_Time"].str.split(":", n = 1, expand = True)[0]

In [34]:
combi.head()

,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Day,Month,Weekday,Hour
0,IndiGo,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info,24,3,Sunday,22
1,Air India,Kolkata,Banglore,05:50,13:15,7h 25m,2 stops,No info,5,1,Saturday,05
2,Jet Airways,Delhi,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info,6,9,Friday,09
3,IndiGo,Kolkata,Banglore,18:05,23:30,5h 25m,1 stop,No info,5,12,Thursday,18
4,IndiGo,Banglore,New Delhi,16:50,21:35,4h 45m,1 stop,No info,3,1,Thursday,16


In [35]:
discount=[]
for i, row in combi.iterrows():
    if row["Hour"]=='00':
        discount.append('Night')
    elif row["Hour"]=='01':
        discount.append('Night')
    elif row["Hour"]=='02':
        discount.append('Night')
    elif row["Hour"]=='03':
        discount.append('Night')
    elif row["Hour"]=='04':
        discount.append('Night')
    elif row["Hour"]=='05':
        discount.append('Night')
    elif row["Hour"]=='06':
        discount.append('Morning')
    elif row["Hour"]=='07':
        discount.append('Morning')
    elif row["Hour"]=='08':
        discount.append('Morning')
    elif row["Hour"]=='09':
        discount.append('Morning')
    elif row["Hour"]=='10':
        discount.append('Morning')
    elif row["Hour"]=='11':
        discount.append('Morning')
    elif row["Hour"]=='12':
        discount.append('Afternoon')
    elif row["Hour"]=='13':
        discount.append('Afternoon')
    elif row["Hour"]=='14':
        discount.append('Afternoon')
    elif row["Hour"]=='15':
        discount.append('Afternoon')
    elif row["Hour"]=='16':
        discount.append('Afternoon')
    elif row["Hour"]=='17':
        discount.append('Afternoon')
    elif row["Hour"]=='18':
        discount.append('Evening')
    elif row["Hour"]=='19':
        discount.append('Evening')
    elif row["Hour"]=='20':
        discount.append('Evening')
    elif row["Hour"]=='21':
        discount.append('Evening')
    elif row["Hour"]=='22':
        discount.append('Evening')
    elif row["Hour"]=='23':
        discount.append('Evening')
    elif row["Hour"]=='24':
        discount.append('Evening')       

In [36]:
len(discount)

13354

In [37]:
combi['Departure']=discount

In [38]:
combi.drop(['Hour'],axis=1,inplace=True)

In [39]:
combi['Departure'].value_counts()

Morning      5345
Evening      3389
Afternoon    3272
Night        1348
Name: Departure, dtype: int64

In [40]:
combi.drop(['Dep_Time'],axis=1,inplace=True)

<h4>Arrival Time</h4>

In [41]:
combi.head()

,Airline,Source,Destination,Arrival_Time,Duration,Total_Stops,Additional_Info,Day,Month,Weekday,Departure
0,IndiGo,Banglore,New Delhi,01:10 22 Mar,2h 50m,non-stop,No info,24,3,Sunday,Evening
1,Air India,Kolkata,Banglore,13:15,7h 25m,2 stops,No info,5,1,Saturday,Night
2,Jet Airways,Delhi,Cochin,04:25 10 Jun,19h,2 stops,No info,6,9,Friday,Morning
3,IndiGo,Kolkata,Banglore,23:30,5h 25m,1 stop,No info,5,12,Thursday,Evening
4,IndiGo,Banglore,New Delhi,21:35,4h 45m,1 stop,No info,3,1,Thursday,Afternoon


In [42]:
#combi['Arrival_Time'] = pd.to_datetime(combi['Arrival_Time'])
#combi=combi.assign(Session_Ar=pd.cut(combi.Arrival_Time.dt.hour,[0,6,12,18,24],labels=['Night','Morning','Afternoon','Evening']))

In [43]:
combi['Hour'] = combi["Arrival_Time"].str.split(":", n = 1, expand = True)[0]

In [44]:
del discount

In [45]:
discount=[]
for i, row in combi.iterrows():
    if row["Hour"]=='00':
        discount.append('Night')
    elif row["Hour"]=='01':
        discount.append('Night')
    elif row["Hour"]=='02':
        discount.append('Night')
    elif row["Hour"]=='03':
        discount.append('Night')
    elif row["Hour"]=='04':
        discount.append('Night')
    elif row["Hour"]=='05':
        discount.append('Night')
    elif row["Hour"]=='06':
        discount.append('Morning')
    elif row["Hour"]=='07':
        discount.append('Morning')
    elif row["Hour"]=='08':
        discount.append('Morning')
    elif row["Hour"]=='09':
        discount.append('Morning')
    elif row["Hour"]=='10':
        discount.append('Morning')
    elif row["Hour"]=='11':
        discount.append('Morning')
    elif row["Hour"]=='12':
        discount.append('Afternoon')
    elif row["Hour"]=='13':
        discount.append('Afternoon')
    elif row["Hour"]=='14':
        discount.append('Afternoon')
    elif row["Hour"]=='15':
        discount.append('Afternoon')
    elif row["Hour"]=='16':
        discount.append('Afternoon')
    elif row["Hour"]=='17':
        discount.append('Afternoon')
    elif row["Hour"]=='18':
        discount.append('Evening')
    elif row["Hour"]=='19':
        discount.append('Evening')
    elif row["Hour"]=='20':
        discount.append('Evening')
    elif row["Hour"]=='21':
        discount.append('Evening')
    elif row["Hour"]=='22':
        discount.append('Evening')
    elif row["Hour"]=='23':
        discount.append('Evening')
    elif row["Hour"]=='24':
        discount.append('Evening')       

In [46]:
combi['Arrival']=discount

In [47]:
combi.drop(['Arrival_Time','Hour'],axis=1,inplace=True)

<h4>Duration</h4>

In [48]:
#Converting Duration into minutes
combi['Duration']=  combi['Duration'].str.replace("h", '*60').str.replace(' ','+').str.replace('m','*1').apply(eval)

In [49]:
combi.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Day,Month,Weekday,Departure,Arrival
0,IndiGo,Banglore,New Delhi,170,non-stop,No info,24,3,Sunday,Evening,Night
1,Air India,Kolkata,Banglore,445,2 stops,No info,5,1,Saturday,Night,Afternoon
2,Jet Airways,Delhi,Cochin,1140,2 stops,No info,6,9,Friday,Morning,Night
3,IndiGo,Kolkata,Banglore,325,1 stop,No info,5,12,Thursday,Evening,Evening
4,IndiGo,Banglore,New Delhi,285,1 stop,No info,3,1,Thursday,Afternoon,Evening


In [50]:
combi.isnull().sum()

Airline            0
Source             0
Destination        0
Duration           0
Total_Stops        0
Additional_Info    0
Day                0
Month              0
Weekday            0
Departure          0
Arrival            0
dtype: int64

Separating Test and Train

In [51]:
train.shape

(10683, 9)

In [52]:
train = combi.iloc[:10683,:]
test = combi.iloc[10683:,:]
y=price

In [53]:
train['Price']=price

In [54]:
test.reset_index(drop=True,inplace=True)

In [55]:
train.isnull().sum()

Airline            0
Source             0
Destination        0
Duration           0
Total_Stops        0
Additional_Info    0
Day                0
Month              0
Weekday            0
Departure          0
Arrival            0
Price              0
dtype: int64

<h4>Mean Encoding</h4>

In [56]:
train.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Day,Month,Weekday,Departure,Arrival,Price
0,IndiGo,Banglore,New Delhi,170,non-stop,No info,24,3,Sunday,Evening,Night,3897
1,Air India,Kolkata,Banglore,445,2 stops,No info,5,1,Saturday,Night,Afternoon,7662
2,Jet Airways,Delhi,Cochin,1140,2 stops,No info,6,9,Friday,Morning,Night,13882
3,IndiGo,Kolkata,Banglore,325,1 stop,No info,5,12,Thursday,Evening,Evening,6218
4,IndiGo,Banglore,New Delhi,285,1 stop,No info,3,1,Thursday,Afternoon,Evening,13302


In [57]:
#Airline
groupbyairline=train.groupby('Airline')['Price'].mean()
train['AirlineMean']=train['Airline'].map(groupbyairline)
test['AirlineMean']=test['Airline'].map(groupbyairline)

In [58]:
#Source
groupbySource=train.groupby('Source')['Price'].mean()
train['SourceMean']=train['Source'].map(groupbySource)
test['SourceMean']=test['Source'].map(groupbySource)

In [59]:
#Destination
groupbyDestination=train.groupby('Destination')['Price'].mean()
train['DestinationMean']=train['Destination'].map(groupbyDestination)
test['DestinationMean']=test['Destination'].map(groupbyDestination)

In [60]:
#Total_Stops
groupbyTotal_Stops=train.groupby('Total_Stops')['Price'].mean()
train['Total_StopsMean']=train['Total_Stops'].map(groupbyTotal_Stops)
test['Total_StopsMean']=test['Total_Stops'].map(groupbyTotal_Stops)

In [61]:
#Additional_Info
groupbyAdditional_Info=train.groupby('Additional_Info')['Price'].mean()
train['Additional_InfoMean']=train['Additional_Info'].map(groupbyAdditional_Info)
test['Additional_InfoMean']=test['Additional_Info'].map(groupbyAdditional_Info)

In [62]:
#Weekday
groupbyWeekday=train.groupby('Weekday')['Price'].mean()
train['WeekdayMean']=train['Weekday'].map(groupbyWeekday)
test['WeekdayMean']=test['Weekday'].map(groupbyWeekday)

In [63]:
#Departure
groupbyDeparture=train.groupby('Departure')['Price'].mean()
train['DepartureMean']=train['Departure'].map(groupbyDeparture)
test['DepartureMean']=test['Departure'].map(groupbyDeparture)

In [64]:
#Arrival
groupbyArrival=train.groupby('Arrival')['Price'].mean()
train['ArrivalMean']=train['Arrival'].map(groupbyArrival)
test['ArrivalMean']=test['Arrival'].map(groupbyArrival)

<h2>Categorical Variables</h2>

In [65]:
train.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Day,Month,Weekday,Departure,Arrival,Price,AirlineMean,SourceMean,DestinationMean,Total_StopsMean,Additional_InfoMean,WeekdayMean,DepartureMean,ArrivalMean
0,IndiGo,Banglore,New Delhi,170,non-stop,No info,24,3,Sunday,Evening,Night,3897,5673.682903,8017.464269,11917.716738,5024.900315,9141.320192,9556.827684,8989.280903,9686.969745
1,Air India,Kolkata,Banglore,445,2 stops,No info,5,1,Saturday,Night,Afternoon,7662,9611.210616,9158.389411,9158.389411,12715.807895,9141.320192,8894.375727,8842.213894,8534.436915
2,Jet Airways,Delhi,Cochin,1140,2 stops,No info,6,9,Friday,Morning,Night,13882,11643.923357,10539.439057,10539.439057,12715.807895,9141.320192,8556.502220,9131.831427,9686.969745
3,IndiGo,Kolkata,Banglore,325,1 stop,No info,5,12,Thursday,Evening,Evening,6218,5673.682903,9158.389411,9158.389411,10593.570032,9141.320192,9805.088646,8989.280903,9590.780790
4,IndiGo,Banglore,New Delhi,285,1 stop,No info,3,1,Thursday,Afternoon,Evening,13302,5673.682903,8017.464269,11917.716738,10593.570032,9141.320192,9805.088646,9217.762289,9590.780790


In [66]:
X=train.drop(['Price'],axis=1)

In [67]:
from sklearn.preprocessing import LabelEncoder
cols=('Airline','Source','Destination','Total_Stops','Additional_Info','Weekday','Departure','Arrival')
for c in cols:
    lbl=LabelEncoder()
    lbl.fit(list(X[c].values))
    X[c]=lbl.transform(list(X[c].values))
    test[c]=lbl.transform(list(test[c].values))

<h1>Modeling</h1>

<h2>AdaBoost Regressor</h2>

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
ada=AdaBoostRegressor(n_estimators=800)

In [ ]:
ada.fit(X,y)

In [ ]:
pred_ada=ada.predict(test)

<h2>GradientBoost Regressor</h2>

In [68]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gb=GradientBoostingRegressor(n_estimators=600)

In [ ]:
gb.fit(X,y)

In [ ]:
pred_gb=gb.predict(test)

<h2>Decision Tree Regressor</h2>

In [69]:
from sklearn.tree import DecisionTreeRegressor

In [70]:
dtr=DecisionTreeRegressor()

In [71]:
dtr.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [72]:
pred_dtr=dtr.predict(test)

<h2>XGBoost</h2>

In [ ]:
import xgboost as xgb

In [ ]:
xgb = xgb.XGBRegressor(n_estimators=500)

In [ ]:
xgb.fit(X,y)

In [ ]:
pred_xgb=xgb.predict(test)

<h2>LightGBM</h2>

In [ ]:
import lightgbm as lgbm

In [ ]:
lgbm=lgbm.LGBMRegressor(n_estimators=700)

In [ ]:
lgbm.fit(X,y)

In [ ]:
pred_lgbm=lgbm.predict(test)

<h3>Random Forest</h3>

In [ ]:
from math import sqrt

In [76]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfc = RandomForestRegressor(n_estimators= 1100, min_samples_split= 15, min_samples_leaf= 1, max_depth= 5)

In [ ]:
rfc.fit(X,y)

In [ ]:
pred_rfc=rfc.predict(test)

In [ ]:
X.head()

<h3>Feature Importance</h3>

In [ ]:
importances = rfc.feature_importances_
std = np.std([tree.feature_importances_ for tree in rfc.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

<h3>StackNet Regressor</h3>

In [ ]:
from pystacknet.pystacknet import StackNetRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,ExtraTreesRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
models=[[KNeighborsRegressor (n_neighbors=300), GradientBoostingRegressor (n_estimators=300),
         ExtraTreesRegressor (n_estimators=300) , DecisionTreeRegressor (),
         AdaBoostRegressor (n_estimators=300)
        ],
        [RandomForestRegressor (n_estimators=400)]
       ]

In [ ]:
model=StackNetRegressor(models, folds=4,restacking=True, n_jobs=1, verbose=1)

In [ ]:
model.fit(X,y)

In [ ]:
predictions_stk=model.predict(test)

<h3>Evaluation Metric</h3>

In [ ]:
def rmsle(y, y_):
   log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
   log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
   calc = (log1 - log2) ** 2
   return np.sqrt(np.mean(calc))

In [ ]:
rmsle(pred_rfc,y)

In [ ]:
y.head()

<h3>Cross Validation</h3>

In [ ]:
rf = RandomForestRegressor()

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Method of selecting samples for training each tree
# bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

pprint(random_grid)


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, np.log(y), test_size=0.2)

In [ ]:
def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = np.sqrt(mean_squared_error(y_test, predictions))
    print('Model Performance')
    print('MSE of: ', errors)
    
    return errors

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
folds = 5
param_comb = 5

skf = KFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(rf, param_distributions=random_grid, n_iter=param_comb, scoring=evaluate, n_jobs=4, cv=skf.split(X,y), verbose=3, random_state=1001 )

In [ ]:
random_search.fit(X, y)

In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)

<h3>Submissions</h3>

In [73]:
pred=pd.DataFrame()

In [74]:
pred['Price']=pred_dtr

In [ ]:
pred['Price']=round(pred['Price'])

In [75]:
pred.to_excel("DecisonTree.xlsx",index=False)

GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_estimators': [75, 100],'max_features':[10, 20],'max_depth':[20, 30]}]
forest_reg = RandomForestRegressor(n_jobs=-1)
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(train_set,train_set_target)